# Choose year

In [ ]:
year = 2020 # See below for available years

## Python imports

In [ ]:
import wget
import pandas as pd
import numpy as np
import unittest
import warnings

## Download data

In [ ]:
years_available = {
  2020: "https://osf.io/qsa4x/download/",
  2021: "https://osf.io/76ufq/download/",
  2022: "https://osf.io/yxvpq/download/"
}

try:
    filename = wget.download(years_available[year])
except:
    Exception('Year not available.')
    
print(filename)

## Generate database

In [ ]:
xls = pd.ExcelFile(filename)

months = [
    'January',
    'February',
    'March',
    'April',
    'May',
    'June',
    'July',
    'August',
    'September',
    'October',
    'November',
    'December'
]

for month in months:
    if month == 'January':
        df = pd.read_excel(xls, month)
    else:
        df = df.append(pd.read_excel(xls, month))


## Cleanup

In [ ]:
# Convert months to proper format
df['Month'] = pd.DatetimeIndex(df['Month']).month

In [ ]:
# Convert languages entries to all lowercase
df['Language(s)'] = df['Language(s)'].str.lower()

In [ ]:
# Convert gender entries to all lowercase
df['First author gender'] = df['First author gender'].str.lower()
df['Last author gender'] = df['Last author gender'].str.lower()

## Global stats

In [ ]:
global_count = df.count()
print(global_count)

In [ ]:
papers_total= global_count['Filename']
print(papers_total)

In [ ]:
keywords_total = global_count['Keywords Matched']
print(keywords_total)

In [ ]:
false_positives = np.sum(df['False Positive?'] == True)
print(false_positives)

In [ ]:
shared_codedata = np.sum(df['False Positive?'] == False)
print(shared_codedata)

## Sanity checks

In [ ]:
if global_count['Keywords Matched'] != global_count['False Positive?']:
    warnings.warn('Keywords matched count must match False positive entries count in database. Correct your Excel sheet.')

In [ ]:
gender_first_author_count = df['First author gender'].value_counts()
gender_last_author_count = df['Last author gender'].value_counts()

In [ ]:
if np.array_equal(gender_first_author_count.keys().tolist(), ['male', 'female']) == False: 
    warnings.warn('Please verify that there is no bad inputs in the first author gender entries')
    print(gender_first_author_count.keys().tolist())

In [ ]:
if np.array_equal(gender_last_author_count.keys().tolist(), ['male', 'female']) == False: 
    warnings.warn('Please verify that there is no bad inputs in the last author gender entries')
    print(gender_last_author_count.keys().tolist())

## Get code/data counts

In [ ]:
shared_code = np.sum(df['Shared code?'] == True)
print(shared_code)

In [ ]:
shared_data = np.sum(df['Shared data?'] == True)
print(shared_data)

## Get website counts

In [ ]:
links_count = df['Link'].value_counts()
print(links_count.to_string())

In [ ]:
github_count = links_count.filter(regex='github')
github_total = np.sum(github_count.values)
print(github_total)

In [ ]:
gitlab_count = links_count.filter(regex='gitlab')
gitlab_total = np.sum(gitlab_count.values)
print(gitlab_total)

In [ ]:
zenodo_count = links_count.filter(regex='zenodo')
zenodo_total = np.sum(zenodo_count.values)
print(zenodo_total)

In [ ]:
osf_count = links_count.filter(regex='osf')
osf_total = np.sum(osf_count.values)
print(osf_total)

## Get language counts

In [ ]:
df['Language(s)'].unique()

In [ ]:
languages_count = df['Language(s)'].value_counts()

In [ ]:
matlab_count = languages_count.filter(regex='matlab')
matlab_total = np.sum(matlab_count.values)
print(matlab_total)

In [ ]:
python_count = languages_count.filter(regex='python')
python_total = np.sum(python_count.values)
print(python_total)

In [ ]:
julia_count = languages_count.filter(regex='julia')
julia_total = np.sum(julia_count.values)
print(julia_total)

In [ ]:
cpp_count = languages_count.filter(regex='c\+\+')
cpp_total = np.sum(cpp_count.values)
print(cpp_total)

In [ ]:
c_count = languages_count.filter(regex='\ c')
c_total = np.sum(c_count.values)
print(c_total)

In [ ]:
shell_count = languages_count.filter(regex='shell')
shell_total = np.sum(shell_count.values)
print(shell_total)

## Get gender count

In [ ]:
first_author_gender_count = df['First author gender'].value_counts()
last_author_gender_count = df['Last author gender'].value_counts()

In [ ]:
male_first = first_author_gender_count['male']
print(male_first)

In [ ]:
female_first = first_author_gender_count['female']
print(female_first)

In [ ]:
male_last = last_author_gender_count['male']
print(male_last)

In [ ]:
female_last = last_author_gender_count['female']
print(female_last)

## Print final statistics

In [ ]:
total_statistics = {}
total_statistics['Total papers'] = papers_total
total_statistics['% papers with matched keyword'] = keywords_total / papers_total * 100
total_statistics['% total papers that did actually share code/data'] = shared_codedata / papers_total * 100
total_statistics['% matched papers that didn\'t actually share code/data'] = false_positives / keywords_total * 100
total_statistics['% matched papers that did actually share code/data'] = shared_codedata / keywords_total * 100
total_statistics['% of total papers that shared code'] = shared_code / papers_total * 100
total_statistics['% of total papers that shared data'] = shared_data / papers_total * 100
total_statistics['% of papers that shared code/data that hosted it on GitHub'] = github_total / shared_codedata * 100
total_statistics['% of papers that shared code that used Python'] = python_total / shared_code * 100
total_statistics['% of papers that shared code that used MATLAB'] = matlab_total / shared_code * 100
total_statistics['% of papers that shared code that used C++'] = cpp_total / shared_code * 100
total_statistics['% of papers that shared code that used Julia'] = julia_total / shared_code * 100
total_statistics['% of papers that shared stuff that had male first authors'] = male_first / shared_codedata * 100
total_statistics['% of papers that shared stuff that had female first authors'] = female_first / shared_codedata * 100
total_statistics['% of papers that shared stuff that had male last authors'] = male_last / shared_codedata * 100
total_statistics['% of papers that shared stuff that had female last authors'] = female_last / shared_codedata * 100

In [ ]:
ser = pd.Series(data=total_statistics)
print(ser.astype(int))

In [ ]:
ser.to_excel("2020.xlsx")